---
# **5. Дивидендные доходности**

### Полная доходность

$$\mu_i^{\text{total}} = \mu_i^{\text{CG}} + \mu_i^{\text{DIV}}$$

где:
- $\mu_i^{\text{CG}}$ — ценовая доходность (Capital Gains)
- $\mu_i^{\text{DIV}}$ — дивидендная доходность

### Ценовая доходность

$$\mu_i^{\text{CG}} = \frac{1}{T-1} \sum_{t=1}^{T-1} r_i(t) \times 252$$

### Дивидендная доходность

$$\mu_i^{\text{DIV}} = \frac{\sum_k d_{i,k}}{\text{avg price}_i} \times \frac{365.25}{\text{period (дней)}}$$

где $d_{i,k}$ — дивиденды, $\text{avg\_price}_i$ — средняя цена.

1. **API MOEX** — автоматический расчёт из истории дивидендов
2. **Ручное задание** — для активов с нестабильной историей
3. **Нулевой доход** — если дивидендов нет

---

In [ ]:
STOCK_SPLITS = {
    'PLZL': pd.Timestamp('2025-03-26'),
}

MANUAL_DIVIDEND_YIELDS = {
    'VTBR': 9.9,
}

YEARS_LOOKBACK = 3

print(f"  Параметры: YEARS={YEARS_LOOKBACK}, РУЧНЫХ={len(MANUAL_DIVIDEND_YIELDS)}, СПЛИТОВ={len(STOCK_SPLITS)}\n")

capital_gains = filtered_returns.mean() * 252

print("Capital Gains (годовые, %):\n")
for ticker in capital_gains.sort_values(ascending=False).index:
    cg = capital_gains[ticker]
    emoji = "🟢" if cg > 0 else "🔴"
    print(f"   {emoji} {ticker:6s}: {cg*100:>7.2f}%")

def get_dividends_yield(ticker, avg_price, end_date, years_back=3):

    if ticker in MANUAL_DIVIDEND_YIELDS:
        return MANUAL_DIVIDEND_YIELDS[ticker] / 100

    try:
        url = f"http://iss.moex.com/iss/securities/{ticker}/dividends.json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        data = response.json()

        if 'dividends' not in data or not data['dividends']['data']:
            return 0.0

        df = pd.DataFrame(data['dividends']['data'], columns=data['dividends']['columns'])
        df['registryclosedate'] = pd.to_datetime(df['registryclosedate'], errors='coerce')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df = df.dropna(subset=['registryclosedate', 'value'])
        df = df[df['value'] > 0]

        if df.empty:
            return 0.0

        if years_back > 0:
            start_date = end_date - pd.DateOffset(years=years_back)
            df = df[(df['registryclosedate'] >= start_date) & (df['registryclosedate'] <= end_date)]

        if df.empty:
            return 0.0

        if ticker in STOCK_SPLITS:
            split_date = STOCK_SPLITS[ticker]
            df_after = df[df['registryclosedate'] >= split_date]
            if not df_after.empty:
                df = df_after

        total_divs = df['value'].sum()
        if avg_price <= 0:
            return 0.0

        date_range = (df['registryclosedate'].max() - df['registryclosedate'].min()).days / 365.25
        if date_range < 0.1:
            date_range = 1.0

        return (total_divs / avg_price) / date_range

    except:
        return 0.0

dividend_yields = {}

for ticker in filtered_names:
    avg_price = priced_df[ticker].mean()
    div_yield = get_dividends_yield(ticker, avg_price, filtered_returns.index.max(), YEARS_LOOKBACK)
    dividend_yields[ticker] = div_yield

    source = "  РУЧНАЯ" if ticker in MANUAL_DIVIDEND_YIELDS else (" API" if div_yield > 0 else "  НЕТ")
    print(f"   {source} {ticker:6s}: {div_yield*100:>6.2f}%")

dividend_yields_series = pd.Series(dividend_yields)

mu_with_dividends = capital_gains + dividend_yields_series

results_df = pd.DataFrame({
    'CG (%)': capital_gains * 100,
    'DIV (%)': dividend_yields_series * 100,
    'μ (%)': mu_with_dividends * 100
}).sort_values('μ (%)', ascending=False)

print(results_df.to_string(float_format='%.2f'))

print(f"\n Статистика:")
print(f"   CG средний:      {capital_gains.mean()*100:>7.2f}%")
print(f"   DIV средний:     {dividend_yields_series.mean()*100:>7.2f}%")
print(f"   μ средний:       {mu_with_dividends.mean()*100:>7.2f}%")
print(f"   Активов с див:   {(dividend_yields_series > 0).sum()}/{len(filtered_names)}")

high_div = dividend_yields_series[dividend_yields_series > 0.05].sort_values(ascending=False)
if not high_div.empty:
    print(f"\n    Высокодивидендные (>5%):")
    for ticker in high_div.index:
        print(f"      {ticker}: {high_div[ticker]*100:.2f}%")

print(f"\n Проверка на аномалии:")

extreme_mu = mu_with_dividends[mu_with_dividends > mu_with_dividends.mean() + 2 * mu_with_dividends.std()]
if not extreme_mu.empty:
    print(f"     Экстремальные доходности (μ > mean+2σ):")
    for ticker in extreme_mu.index:
        print(f"      {ticker}: {extreme_mu[ticker]*100:.2f}%")
else:
    print(f"    Аномалий не обнаружено")

mu_conservative = mu_with_dividends * 0.8
print(f"   Консервативные μ (80% от текущих):")
for ticker in mu_conservative.sort_values(ascending=False).head(5).index:
    print(f"      {ticker}: {mu_conservative[ticker]*100:.2f}%")

print(f" Переменные:")
print(f"   mu_with_dividends      : ({len(mu_with_dividends)},) — итоговые доходности")
print(f"   dividend_yields_series : ({len(dividend_yields_series)},) — дивидендные доходности")
print(f"   capital_gains          : ({len(capital_gains)},) — ценовые доходности")
print(f"   mu_conservative        : ({len(mu_conservative)},) — консервативные доходности\n")